# Data Preprocessing in Python

## Importing Dependencies and uploading our data

In [0]:
import pandas as pd
import numpy as np
from google.colab import files
import math

In [0]:
data = files.upload()

### Sanity check to ensure our data was successfully uploaded

In [0]:
!ls

mock_data_preprocessing.csv  sample_data


## Data Exploration

In [0]:
dataset = pd.read_csv('mock_data_preprocessing.csv')

In [288]:
dataset.head(9)

,Id,Age,Siblings,Height,Weight,BMI,Vehicle,Travel Distance,Education,Salary
0,x1,23,None,5’5,140lbs,23.3,Car,60km,Bachelor’s,"$60,000"
1,x2,20,2,6’0,175lbs,23.7,Bicycle,20km,High-School,"$20,000"
2,x3,40,1,4’10,100lbs,999.0,Bus,40km,Master’s,"$75,000"
3,x4,21,6,0,130lbs,23.8,Taxi,50km,High-School,"$2,000,000"
4,x5,42,0,5’8,180,27.4,Bus,35km,Ph.D.,"$80,000"
5,x5,42,0,5’8,180,27.4,Bus,35km,Ph.D.,"$80,000"
6,x1,23,None,5’5,140lbs,23.3,Car,60km,Bachelor’s,"$60,000"
7,x7,24,1,5’8,155lbs,23.6,Foot,3km,Bachelor’s,"$100,000"
8,x8,35,4,5’3,100lbs,17.7,Bus,15km,Master’s,"$50,000"


### Note:
.describe() is only showing us the Age and BMI columns because they are the only columns with only numbers in them. This is a sign that all the other columns will need to be processed to some degree as well.

In [289]:
print(dataset.describe())
print('============================')
print(dataset.dtypes)

             Age         BMI
count   9.000000    9.000000
mean   30.000000  132.133333
std     9.539392  325.087242
min    20.000000   17.700000
25%    23.000000   23.300000
50%    24.000000   23.700000
75%    40.000000   27.400000
max    42.000000  999.000000
Id                  object
Age                  int64
Siblings            object
Height              object
Weight              object
BMI                float64
Vehicle             object
Travel Distance     object
Education           object
Salary              object
dtype: object


## Data Preprocessing

### Duplicates

To reduce the amount of computation required to clean the data we will first remove any existing duplicates.

**TODO**

1.) Check if there are any duplicate rows

2.) Remove any duplicate rows

**Note:** Make sure to keep one copy of any duplicates found in the dataset

In [0]:
dataset=dataset.drop_duplicates()

### Age

In order to prepare our data for training we will normalize all our data to be between 0 and 1.

**Note:** When normalizing data only look at the values of the feature/column being normalized

**TODO**

1.) Find the minimum age

2.) Find the maximum age

3.) Normalize all the age values

**Note**

The normalize formula is as follows:

$$ X_{new} = \frac{X - X_{min}}  {X_{max} - X_{min}}$$



In [0]:
minimum=dataset.Age.min()
maxi=dataset.Age.max()
dataset.Age=dataset.Age.apply(lambda x:(x-minimum)/(maxi-minimum))

In [292]:
dataset

,Id,Age,Siblings,Height,Weight,BMI,Vehicle,Travel Distance,Education,Salary
0,x1,0.136364,None,5’5,140lbs,23.3,Car,60km,Bachelor’s,"$60,000"
1,x2,0.000000,2,6’0,175lbs,23.7,Bicycle,20km,High-School,"$20,000"
2,x3,0.909091,1,4’10,100lbs,999.0,Bus,40km,Master’s,"$75,000"
3,x4,0.045455,6,0,130lbs,23.8,Taxi,50km,High-School,"$2,000,000"
4,x5,1.000000,0,5’8,180,27.4,Bus,35km,Ph.D.,"$80,000"
7,x7,0.181818,1,5’8,155lbs,23.6,Foot,3km,Bachelor’s,"$100,000"
8,x8,0.681818,4,5’3,100lbs,17.7,Bus,15km,Master’s,"$50,000"


### Siblings

Under Siblings we notice that we have a Value of "None". This could be due to an error or it could mean that they have no siblings. Either way we need to convert this to a number.

**Todo**

1.) Replace any None values in Siblings with 0

2.) Find the minimum value

3.) Find the maximum value

4.) Normalize the siblings values

**Note**

Only numerical data types(ints, floats) can be normalized.

In [0]:
dataset['Siblings']=dataset['Siblings'].astype(str)
dataset.loc[dataset['Siblings'] == 'None', 'Siblings'] = '0'

In [0]:
dataset['Siblings']=dataset['Siblings'].astype(int)
dataset.Siblings=dataset.Siblings.apply(lambda x:(x-dataset.Siblings.min())/(dataset.Siblings.max()-dataset.Siblings.min()))

In [295]:
dataset

,Id,Age,Siblings,Height,Weight,BMI,Vehicle,Travel Distance,Education,Salary
0,x1,0.136364,0.000000,5’5,140lbs,23.3,Car,60km,Bachelor’s,"$60,000"
1,x2,0.000000,0.333333,6’0,175lbs,23.7,Bicycle,20km,High-School,"$20,000"
2,x3,0.909091,0.166667,4’10,100lbs,999.0,Bus,40km,Master’s,"$75,000"
3,x4,0.045455,1.000000,0,130lbs,23.8,Taxi,50km,High-School,"$2,000,000"
4,x5,1.000000,0.000000,5’8,180,27.4,Bus,35km,Ph.D.,"$80,000"
7,x7,0.181818,0.166667,5’8,155lbs,23.6,Foot,3km,Bachelor’s,"$100,000"
8,x8,0.681818,0.666667,5’3,100lbs,17.7,Bus,15km,Master’s,"$50,000"


### Height, Weight and BMI
The BMI column allows us to deal with these three columns at the same time. We can use the following formula to fix any missing values as well:

$$BMI = \frac{703 * Weight_{pounds}}  {Height_{inches}^2} $$

**TODO**

1.) Convert the Height values to inches

2.) Remove 'lbs' from the Weight values

3.) Find and fix the outlier in the BMI column

4.) Find and fix the 0 value in Height

5.) Normalize all three columns

In [0]:
dataset.Height=dataset.Height.astype(str)
def split(ht):
  if len(ht)>1:
    hei = ht.split("’")
    feet= float(hei[0])
    inch = float(hei[1])
    return (12*feet) + inch

dataset.Height=dataset["Height"].apply(lambda x:split(x))

In [0]:
dataset.Weight=dataset.Weight.astype(str)
dataset.Weight=dataset["Weight"].apply(lambda x:x.rstrip('lbs'))
dataset.Weight=dataset.Weight.astype(int)

In [298]:
def BMI(hei,wei):
  return (703*wei/hei**2)
dataset.BMI.loc[2]=BMI(dataset.Height.loc[2],dataset.Weight.loc[2])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [299]:
def height(bmi,wei):
  return ((703*wei/bmi)**0.5)
dataset.Height.loc[3]=height(dataset.BMI.loc[3],dataset.Weight.loc[3])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
def normalize(x,col):
  return (x-dataset[col].min())/(dataset[col].max()-dataset[col].min())
dataset.Height=dataset.Height.apply(lambda x: normalize(x,'Height'))
dataset.Weight=dataset.Weight.apply(lambda x: normalize(x,'Weight'))
dataset.BMI=dataset.BMI.apply(lambda x: normalize(x,'BMI'))

In [301]:
dataset

,Id,Age,Siblings,Height,Weight,BMI,Vehicle,Travel Distance,Education,Salary
0,x1,0.136364,0.000000,0.500000,0.5000,0.577320,Car,60km,Bachelor’s,"$60,000"
1,x2,0.000000,0.333333,1.000000,0.9375,0.618557,Bicycle,20km,High-School,"$20,000"
2,x3,0.909091,0.166667,0.000000,0.0000,0.329664,Bus,40km,Master’s,"$75,000"
3,x4,0.045455,1.000000,0.283361,0.3750,0.628866,Taxi,50km,High-School,"$2,000,000"
4,x5,1.000000,0.000000,0.714286,1.0000,1.000000,Bus,35km,Ph.D.,"$80,000"
7,x7,0.181818,0.166667,0.714286,0.6875,0.608247,Foot,3km,Bachelor’s,"$100,000"
8,x8,0.681818,0.666667,0.357143,0.0000,0.000000,Bus,15km,Master’s,"$50,000"


### Vehicle
The entries in the Vehicle column each have text representing that cell's value. Since ML models require numerical data to work, we will need to encode the categorical values into a numerical form. One-Hot encoding is a popular way of dealing with categorical data.

One-Hot encoding works by creating multiple boolean variables and assigning each variable to one category.

**ex.** If our categories were Cat, Dog, Bird. We can encode them as followed: Cat = 1 0 0,  Dog = 0 1 0, Bird = 0 0 1
<br>

**TODO**

1.) Create a copy of the Vehicle column

2.) Convert the copy into a One-Hot representation

3.) Delete the original Vehicle column

4.) Add the One-Hot encoded representation back into the dataset

In [0]:
copy=dataset.Vehicle
OneHot={
    'Car':[0,0,0],
    'Bicycle':[0,0,1],
    'Bus':[0,1,0],
    'Foot':[0,1,1],
    'Taxi':[1,0,0]
}
copy=copy.map(OneHot)
del dataset['Vehicle']
dataset['Vehicule']=copy.values

In [303]:
dataset

,Id,Age,Siblings,Height,Weight,BMI,Travel Distance,Education,Salary,Vehicule
0,x1,0.136364,0.000000,0.500000,0.5000,0.577320,60km,Bachelor’s,"$60,000","[0, 0, 0]"
1,x2,0.000000,0.333333,1.000000,0.9375,0.618557,20km,High-School,"$20,000","[0, 0, 1]"
2,x3,0.909091,0.166667,0.000000,0.0000,0.329664,40km,Master’s,"$75,000","[0, 1, 0]"
3,x4,0.045455,1.000000,0.283361,0.3750,0.628866,50km,High-School,"$2,000,000","[1, 0, 0]"
4,x5,1.000000,0.000000,0.714286,1.0000,1.000000,35km,Ph.D.,"$80,000","[0, 1, 0]"
7,x7,0.181818,0.166667,0.714286,0.6875,0.608247,3km,Bachelor’s,"$100,000","[0, 1, 1]"
8,x8,0.681818,0.666667,0.357143,0.0000,0.000000,15km,Master’s,"$50,000","[0, 1, 0]"


### Travel Distance

Just as with Weights column, we need to remove the units and normalize the values.

**TODO**

1.) Remove the km unit from all the values in the Travel Distance column

2.) Normalize all the values in the columns

In [0]:

dataset["Travel Distance"]=dataset["Travel Distance"].apply(lambda x:x.rstrip('km'))
dataset["Travel Distance"]=dataset["Travel Distance"].astype(int)
dataset["Travel Distance"]=dataset["Travel Distance"].apply(lambda x: normalize(x,"Travel Distance"))

In [305]:
dataset

,Id,Age,Siblings,Height,Weight,BMI,Travel Distance,Education,Salary,Vehicule
0,x1,0.136364,0.000000,0.500000,0.5000,0.577320,1.000000,Bachelor’s,"$60,000","[0, 0, 0]"
1,x2,0.000000,0.333333,1.000000,0.9375,0.618557,0.298246,High-School,"$20,000","[0, 0, 1]"
2,x3,0.909091,0.166667,0.000000,0.0000,0.329664,0.649123,Master’s,"$75,000","[0, 1, 0]"
3,x4,0.045455,1.000000,0.283361,0.3750,0.628866,0.824561,High-School,"$2,000,000","[1, 0, 0]"
4,x5,1.000000,0.000000,0.714286,1.0000,1.000000,0.561404,Ph.D.,"$80,000","[0, 1, 0]"
7,x7,0.181818,0.166667,0.714286,0.6875,0.608247,0.000000,Bachelor’s,"$100,000","[0, 1, 1]"
8,x8,0.681818,0.666667,0.357143,0.0000,0.000000,0.210526,Master’s,"$50,000","[0, 1, 0]"


### Education

Once again we are dealing with text values in the Education column. This time though since the categories can be ordered,  we are dealing with ordinal data instead. This allows us to encode the data differently in order to limit the number of columns in our dataset.

Instead of using One-Hot encoding which will have us add k-1 columns to our dataset, we will encode all the categories in one column.

**Ex.:** If our data was: Beginner, Intermediate, Advance. Instead of encoding as One-Hot, we can encode it as followed: Beginner = 0, Intermediate = 1, Advance = 2.

**Note:** There are still issues with encoding data like this as we are saying that the distance between each category is the same. Depending on the data this may not be true.

 **TODO**
 
 1.) Create a mapping from category to number. Keep in mind that since we are dealing with ordinal data we want to preserve the order of the values after the encoding
 
 2.) Use the mapping to convert the values in the Education column into numbers
 
 3.) Normalize the values

In [0]:
OneHot={
    'High-School':0,
    'Bachelor’s':1,
    'Master’s':2,
    'Ph.D.':3,
}
dataset.Education=dataset.Education.map(OneHot)

dataset["Education"]=dataset["Education"].apply(lambda x: normalize(x,"Education"))

In [307]:
dataset

,Id,Age,Siblings,Height,Weight,BMI,Travel Distance,Education,Salary,Vehicule
0,x1,0.136364,0.000000,0.500000,0.5000,0.577320,1.000000,0.333333,"$60,000","[0, 0, 0]"
1,x2,0.000000,0.333333,1.000000,0.9375,0.618557,0.298246,0.000000,"$20,000","[0, 0, 1]"
2,x3,0.909091,0.166667,0.000000,0.0000,0.329664,0.649123,0.666667,"$75,000","[0, 1, 0]"
3,x4,0.045455,1.000000,0.283361,0.3750,0.628866,0.824561,0.000000,"$2,000,000","[1, 0, 0]"
4,x5,1.000000,0.000000,0.714286,1.0000,1.000000,0.561404,1.000000,"$80,000","[0, 1, 0]"
7,x7,0.181818,0.166667,0.714286,0.6875,0.608247,0.000000,0.333333,"$100,000","[0, 1, 1]"
8,x8,0.681818,0.666667,0.357143,0.0000,0.000000,0.210526,0.666667,"$50,000","[0, 1, 0]"


### Salary
The values in the Salary column contain special characters that we need to remove in order for our models to work

**TODO**

1.) Remove all the special characters

2.) Deal with any outliers

3.) Normalize the data

In [0]:
dataset.Salary=dataset.Salary.astype(str)
dataset.Salary=dataset["Salary"].apply(lambda x: x.lstrip('$,'))
dataset.Salary=dataset["Salary"].str.replace(',','')
dataset.Salary=dataset.Salary.astype(int)
dataset["Salary"]=dataset["Salary"].apply(lambda x: normalize(x,"Salary"))


In [309]:
dataset

,Id,Age,Siblings,Height,Weight,BMI,Travel Distance,Education,Salary,Vehicule
0,x1,0.136364,0.000000,0.500000,0.5000,0.577320,1.000000,0.333333,0.020202,"[0, 0, 0]"
1,x2,0.000000,0.333333,1.000000,0.9375,0.618557,0.298246,0.000000,0.000000,"[0, 0, 1]"
2,x3,0.909091,0.166667,0.000000,0.0000,0.329664,0.649123,0.666667,0.027778,"[0, 1, 0]"
3,x4,0.045455,1.000000,0.283361,0.3750,0.628866,0.824561,0.000000,1.000000,"[1, 0, 0]"
4,x5,1.000000,0.000000,0.714286,1.0000,1.000000,0.561404,1.000000,0.030303,"[0, 1, 0]"
7,x7,0.181818,0.166667,0.714286,0.6875,0.608247,0.000000,0.333333,0.040404,"[0, 1, 1]"
8,x8,0.681818,0.666667,0.357143,0.0000,0.000000,0.210526,0.666667,0.015152,"[0, 1, 0]"


## Our Clean Data

In [310]:

dataset.head(7)

,Id,Age,Siblings,Height,Weight,BMI,Travel Distance,Education,Salary,Vehicule
0,x1,0.136364,0.000000,0.500000,0.5000,0.577320,1.000000,0.333333,0.020202,"[0, 0, 0]"
1,x2,0.000000,0.333333,1.000000,0.9375,0.618557,0.298246,0.000000,0.000000,"[0, 0, 1]"
2,x3,0.909091,0.166667,0.000000,0.0000,0.329664,0.649123,0.666667,0.027778,"[0, 1, 0]"
3,x4,0.045455,1.000000,0.283361,0.3750,0.628866,0.824561,0.000000,1.000000,"[1, 0, 0]"
4,x5,1.000000,0.000000,0.714286,1.0000,1.000000,0.561404,1.000000,0.030303,"[0, 1, 0]"
7,x7,0.181818,0.166667,0.714286,0.6875,0.608247,0.000000,0.333333,0.040404,"[0, 1, 1]"
8,x8,0.681818,0.666667,0.357143,0.0000,0.000000,0.210526,0.666667,0.015152,"[0, 1, 0]"


In [311]:
print(dataset.describe())
print('============================')
print(dataset.dtypes)

            Age  Siblings    Height  ...  Travel Distance  Education    Salary
count  7.000000  7.000000  7.000000  ...         7.000000   7.000000  7.000000
mean   0.422078  0.333333  0.509868  ...         0.506266   0.428571  0.161977
std    0.427784  0.372678  0.331284  ...         0.354865   0.370899  0.369753
min    0.000000  0.000000  0.000000  ...         0.000000   0.000000  0.000000
25%    0.090909  0.083333  0.320252  ...         0.254386   0.166667  0.017677
50%    0.181818  0.166667  0.500000  ...         0.561404   0.333333  0.027778
75%    0.795455  0.500000  0.714286  ...         0.736842   0.666667  0.035354
max    1.000000  1.000000  1.000000  ...         1.000000   1.000000  1.000000

[8 rows x 8 columns]
Id                  object
Age                float64
Siblings           float64
Height             float64
Weight             float64
BMI                float64
Travel Distance    float64
Education          float64
Salary             float64
Vehicule            obje